🧠 КАК ДУМАТЬ, КОГДА СМОТРИШЬ НА ДАННЫЕ

Это не шпора по командам — это шпора по **мышлению**.

Цель: научиться задавать правильные вопросы данным.

---
ПЕРВЫЕ 5 МИНУТ: Что делать сразу после загрузки данных?

**Вопросы, которые нужно задать себе:**

1. **Что я предсказываю?** (таргет)
2. **Сколько у меня данных?** (достаточно ли для ML?)
3. **Какие признаки есть?** (числа, категории, даты, текст?)
4. **Есть ли пропуски?** (много ли? в каких колонках?)
5. **Есть ли очевидные проблемы?** (дубликаты, странные значения)

In [ ]:
import pandas as pd
import numpy as np

# Загрузка
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

# === ПЕРВЫЙ ВЗГЛЯД ===
print('📊 РАЗМЕРЫ:')
print(f'Train: {train.shape[0]} строк, {train.shape[1]} колонок')
print(f'Test: {test.shape[0]} строк, {test.shape[1]} колонок')
print()

print('👀 ПЕРВЫЕ СТРОКИ:')
display(train.head())

In [ ]:
# === ЧТО ЗА ДАННЫЕ? ===
print('📋 ТИПЫ ДАННЫХ:')
print(train.dtypes)
print()

print('🔢 ЧИСЛОВЫЕ КОЛОНКИ:', train.select_dtypes(include=['int64', 'float64']).columns.tolist())
print('📝 КАТЕГОРИАЛЬНЫЕ:', train.select_dtypes(include=['object']).columns.tolist())

In [ ]:
# === ПРОПУСКИ ===
print('❓ ПРОПУСКИ:')
missing = train.isnull().sum()
missing_pct = (missing / len(train) * 100).round(1)
missing_df = pd.DataFrame({'Пропусков': missing, '%': missing_pct})
print(missing_df[missing_df['Пропусков'] > 0].sort_values('%', ascending=False))

if missing.sum() == 0:
    print('Пропусков нет! 🎉')

---
СМОТРИМ НА ТАРГЕТ: Это первое, что нужно понять!

**Вопросы:**
- Это **классификация** или **регрессия**?
- Если классификация — **сбалансированы ли классы**?
- Если регрессия — **какое распределение** у таргета? (нормальное? есть выбросы?)

In [ ]:
TARGET = 'target'  # <-- ПОМЕНЯЙ

print('🎯 ТАРГЕТ:')
print(f'Уникальных значений: {train[TARGET].nunique()}')
print()

# Если классификация (мало уникальных значений)
if train[TARGET].nunique() <= 10:
    print('Это КЛАССИФИКАЦИЯ')
    print('Распределение классов:')
    print(train[TARGET].value_counts())
    print()
    print('В процентах:')
    print((train[TARGET].value_counts(normalize=True) * 100).round(1))
    
    # Проверка баланса
    balance = train[TARGET].value_counts(normalize=True).min()
    if balance < 0.1:
        print('\n⚠️ ВНИМАНИЕ: Классы сильно несбалансированы!')
        print('Подумай о class_weight или стратифицированной выборке')
else:
    print('Это РЕГРЕССИЯ')
    print(train[TARGET].describe())

In [ ]:
import matplotlib.pyplot as plt

# Визуализация таргета
if train[TARGET].nunique() <= 10:
    train[TARGET].value_counts().plot(kind='bar')
    plt.title('Распределение классов')
else:
    train[TARGET].hist(bins=50)
    plt.title('Распределение таргета')
plt.show()

---
СМОТРИМ НА ПРИЗНАКИ: Ищем полезное и проблемное

**Что искать:**

- **Выбросы** — есть ли аномальные значения?
- **Константные признаки** — колонки с одним значением (бесполезны)
- **Высокая кардинальность** — категории с 1000+ значений (могут быть проблемой)
- **Корреляции с таргетом** — какие признаки важны?

In [ ]:
# === СТАТИСТИКА ПО ЧИСЛОВЫМ ===
print('📈 СТАТИСТИКА ЧИСЛОВЫХ ПРИЗНАКОВ:')
display(train.describe())

In [ ]:
# === ПРОВЕРКА НА ПРОБЛЕМНЫЕ ПРИЗНАКИ ===

print('🚨 ПРОВЕРКА НА ПРОБЛЕМЫ:')
print()

# 1. Константные признаки
const_cols = [c for c in train.columns if train[c].nunique() == 1]
if const_cols:
    print(f'❌ Константные колонки (удалить!): {const_cols}')
else:
    print('✅ Константных колонок нет')

# 2. Почти константные (>99% одно значение)
almost_const = []
for c in train.columns:
    if train[c].value_counts(normalize=True).iloc[0] > 0.99:
        almost_const.append(c)
if almost_const:
    print(f'⚠️ Почти константные: {almost_const}')

# 3. Высокая кардинальность
high_card = [c for c in train.select_dtypes(include=['object']).columns 
             if train[c].nunique() > 100]
if high_card:
    print(f'⚠️ Высокая кардинальность: {high_card}')
    print('   (много уникальных значений — может быть проблемой)')

In [ ]:
# === КОРРЕЛЯЦИИ С ТАРГЕТОМ ===
# Работает только для числовых признаков

if train[TARGET].dtype in ['int64', 'float64']:
    correlations = train.corr(numeric_only=True)[TARGET].drop(TARGET).sort_values(key=abs, ascending=False)
    print('🔗 КОРРЕЛЯЦИИ С ТАРГЕТОМ (топ-10):')
    print(correlations.head(10))
    print()
    print('💡 ВЫВОД: Эти признаки скорее всего важны для модели')

---
ИЩЕМ СВЯЗИ: Как признаки связаны друг с другом?

**Зачем это нужно:**
- Найти **лики** (data leakage) — признаки, которые "подсматривают" в ответ
- Понять, какие признаки можно **объединить**
- Найти **мультиколлинеарность** — когда признаки дублируют друг друга

In [ ]:
import seaborn as sns

# Тепловая карта корреляций
plt.figure(figsize=(12, 10))
corr_matrix = train.select_dtypes(include=['int64', 'float64']).corr()
sns.heatmap(corr_matrix, annot=True, fmt='.2f', cmap='coolwarm', center=0)
plt.title('Корреляции между признаками')
plt.tight_layout()
plt.show()

In [ ]:
# СМ ВЫШЕ
# correlations = train.corr(numeric_only=True)[TARGET].drop(TARGET).sort_values(key=abs, ascending=False)

# === ПОИСК ПОТЕНЦИАЛЬНЫХ ЛИКОВ ===

if train[TARGET].dtype in ['int64', 'float64']:
    high_corr = correlations[abs(correlations) > 0.9]
    if len(high_corr) > 0:
        print('🚨 ВНИМАНИЕ! Очень высокая корреляция с таргетом:')
        print(high_corr)
        print()
        print('Это может быть:')
        print('1. DATA LEAKAGE — признак "знает" ответ (плохо!)')
        print('2. Очень хороший признак (хорошо, но проверь)')
        print()
        print('Как проверить: посмотри, мог ли этот признак быть известен ДО того, как стал известен таргет?')

---
ВЫБРОСЫ: Как их найти и что с ними делать

In [ ]:
# === ПОИСК ВЫБРОСОВ ===

def find_outliers(df, col):
    """Находит выбросы по методу IQR"""
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower) | (df[col] > upper)]
    return len(outliers), lower, upper

num_cols = train.select_dtypes(include=['int64', 'float64']).columns.tolist()

print('🔍 ВЫБРОСЫ:')
for col in num_cols:
    n_outliers, lower, upper = find_outliers(train, col)
    if n_outliers > 0:
        pct = n_outliers / len(train) * 100
        print(f'{col}: {n_outliers} выбросов ({pct:.1f}%)')

**Что делать с выбросами?**

| Ситуация | Действие |
|----------|----------|
| Явная ошибка данных | Удалить или заменить на NaN |
| Реальные редкие случаи | Оставить, модель справится |
| Много выбросов (>5%) | Подумай о winsorization (обрезать хвосты) |

**Важно:** Бустинги (CatBoost, LightGBM) обычно хорошо справляются с выбросами!

---
GROUPBY: Главная команда для понимания данных

**Groupby отвечает на вопросы:**
- Как среднее значение X отличается для разных групп?
- В какой группе больше всего Y?
- Как распределён таргет по категориям?

In [ ]:
# === GROUPBY ПРИМЕРЫ ===

# Если есть категориальный признак, посмотрим средний таргет по группам
cat_col = cat_cols[0] if cat_cols else None  # берём первый категориальный

if cat_col and train[TARGET].dtype in ['int64', 'float64']:
    print(f'📊 Средний {TARGET} по группам {cat_col}:')
    grouped = train.groupby(cat_col)[TARGET].agg(['mean', 'count', 'std'])
    grouped = grouped.sort_values('mean', ascending=False)
    print(grouped)
    print()
    print('💡 Если средние сильно отличаются — этот признак важен!')

In [ ]:
# === ПОЛЕЗНЫЕ GROUPBY ПАТТЕРНЫ ===

# 1. Среднее по группам
# train.groupby('category')['value'].mean()

# 2. Несколько статистик сразу
# train.groupby('category')['value'].agg(['mean', 'median', 'std', 'count'])

# 3. Группировка по нескольким колонкам
# train.groupby(['cat1', 'cat2'])['value'].mean()

# 4. Применить разные функции к разным колонкам
# train.groupby('category').agg({'col1': 'mean', 'col2': 'sum', 'col3': 'count'})

# 5. Transform — вернуть результат той же длины (для feature engineering!)
# train['group_mean'] = train.groupby('category')['value'].transform('mean')

---
FEATURE ENGINEERING НА ОСНОВЕ EDA

**Идеи признаков на основе того, что мы увидели:**

In [ ]:
# === FEATURE ENGINEERING ===

def smart_fe(df, cat_cols, num_cols):
    """Умный feature engineering на основе EDA"""
    df = df.copy()
    
    # 1. Агрегаты по группам (target encoding lite)
    for cat in cat_cols[:3]:  # топ-3 категориальных
        for num in num_cols[:3]:  # топ-3 числовых
            df[f'{cat}_{num}_mean'] = df.groupby(cat)[num].transform('mean')
    
    # 2. Отношения между числовыми
    if len(num_cols) >= 2:
        df['ratio_0_1'] = df[num_cols[0]] / (df[num_cols[1]] + 1)
        df['diff_0_1'] = df[num_cols[0]] - df[num_cols[1]]
    
    # 3. Счётчик редких категорий
    for cat in cat_cols:
        counts = df[cat].value_counts()
        df[f'{cat}_count'] = df[cat].map(counts)
        df[f'{cat}_is_rare'] = (df[f'{cat}_count'] < 10).astype(int)
    
    return df

# Применяем
# train = smart_fe(train, cat_cols, num_cols)
# test = smart_fe(test, cat_cols, num_cols)

---
ЧЕКЛИСТ: Что проверить перед обучением модели

✅ Понял, что предсказываю (классификация/регрессия)  
✅ Проверил баланс классов  
✅ Посмотрел пропуски  
✅ Нашёл корреляции с таргетом  
✅ Проверил на лики (слишком высокая корреляция)  
✅ Посмотрел категориальные признаки  
✅ Проверил, что в test нет новых категорий  
✅ Подумал о feature engineering

---
ДОКУМЕНТАЦИЯ БЕЗ ГУГЛА

Как смотреть документацию прямо в Jupyter:

In [ ]:
# Способ 1: Знак вопроса
# pd.read_csv?

# Способ 2: Два знака вопроса (исходный код)
# pd.read_csv??

# Способ 3: help()
# help(pd.read_csv)

# Способ 4: dir() — список всех методов объекта
# dir(train)  # все методы DataFrame
# [m for m in dir(train) if not m.startswith('_')]  # без приватных